<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ロジスティック回帰による異常検知の実装

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay
import matplotlib.cm as cm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")

## 学習

In [ ]:
# 異常と正常を判別するロジスティック回帰モデルの学習
# "temp1", "temp2"変数に欠損があるデータを削除
df_dropna = df.dropna(subset=["temp2", "temp1"])
# データをnumpy.ndarrayに変換
X = df_dropna[["temp2", "temp1"]].to_numpy()
y = df_dropna["label"].to_numpy()
# ロジスティック回帰モデルの学習
clf = LogisticRegression(penalty=None)
clf.fit(X, y)
# 決定境界をプロット
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
disp = DecisionBoundaryDisplay.from_estimator(
    clf, X, response_method="predict",
    xlabel="temp2", ylabel="temp1",
    cmap=cm.gray, alpha=0.5, ax=ax
)
# 散布図を重ねてプロット
sns.scatterplot(
    data=df_dropna, x="temp2", y="temp1",
    hue="label", palette=["#999999", "#111111"],
    ax=ax
)
plt.show()

## 推論

In [ ]:
df_inference = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_inference.csv")

In [ ]:
# ロジスティック回帰モデルの推論
# "temp2", "temp1"変数に欠損があるデータの削除
df_inference_dropna = df_inference.dropna(subset=["temp2", "temp1"])
# データから2変数のみ取り出してnumpy.ndarrayに変換
X_inference = df_inference_dropna[["temp2", "temp1"]].to_numpy()
y_inference = df_inference_dropna["label"].to_numpy()
# 学習済みロジスティック回帰モデルで推論
y_pred = clf.predict(X_inference)
# 推論結果を表示
print(y_pred)

### クラス確率のプロット

In [ ]:
import numpy as np

# クラス確率描画用のメッシュデータを作成
temp2_min, temp2_max = np.min(X[:, 0]), np.max(X[:, 0])
temp1_min, temp1_max = np.min(X[:, 1]), np.max(X[:, 1])
temp2_axis = np.linspace(temp2_min, temp2_max, num=100)
temp1_axis = np.linspace(temp1_min, temp1_max, num=100)
X_temp2, X_temp1 = np.meshgrid(temp2_axis, temp1_axis)
X_grid = np.c_[X_temp2.ravel(), X_temp1.ravel()]
# 学習済みモデルからクラス確率を取得
y_proba_grid = clf.predict_proba(X_grid)
# 異常確率を抽出してピボット化
anomaly_proba_pivot = y_proba_grid[:, 0].reshape(X_temp2.shape)
# クラス確率をプロット
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
ax.contourf(X_temp2, X_temp1, anomaly_proba_pivot, levels=50, cmap="Greys")
# 散布図を重ねてプロット
sns.scatterplot(data=df_dropna, x="temp2", y="temp1",
                hue="label", palette=["#999999", "#111111"],
                ax=ax)
plt.show()

- 決定境界付近でクラス確率が急激に変化していることがわかる

In [ ]:
X_temp2.ravel().shape

## ロジスティック回帰以外の手法でのクラス確率の予測
- SVMでのクラス確率の予測

In [ ]:
# SVMでのクラス確率の予測
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# SVMモデルを学習（probability=Trueを設定する必要がある）
svm = SVC(C=10, gamma=0.1, probability=True)
clf = Pipeline([("scaler", StandardScaler()), ("svm", svm)])
clf.fit(X, y)

# 学習済みモデルからクラス確率を取得
y_proba_grid = clf.predict_proba(X_grid)
# 異常確率を抽出してピボット化
anomaly_proba_pivot = y_proba_grid[:, 0].reshape(X_temp2.shape)
# クラス確率をプロット
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
ax.contourf(X_temp2, X_temp1, anomaly_proba_pivot, levels=50, cmap="Greys")
# 散布図を重ねてプロット
sns.scatterplot(data=df_dropna, x="temp2", y="temp1", hue="label",
                palette=["#999999", "#111111"], ax=ax)
plt.show()

- ロジスティック回帰と比べて、SVMはより曲線的なクラス確率分布を持つ
- カーネルトリックにより、曲線的な決定境界が引かれることに由来する